In [1]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster

In [2]:
# Define o nome do arquivo CSV a ser lido
filename = 'Dataset.csv'

# Lê o arquivo CSV, usando o ponto e vírgula como separador e a coluna 'Id' como índice, remove linhas onde 'Cluster', 'Latitude' ou 'Longitude' estejam faltando (NaN) e converte a coluna 'Cluster' para o tipo inteiro de 64 bits (np.int64)
df0 = pd.read_csv(filename, sep=';', index_col='Id').dropna(subset=['Cluster', 'Latitude', 'Longitude']).astype({'Cluster': np.int64})

# Exibe o DataFrame resultante
df0

,Latitude,Longitude,Cluster
Id,,,
0003fb6a-bca5-45bd-b7c1-919ee1516a5e,-7.135327,-34.855608,3
001c3406-3775-40fb-b1eb-4829d0ed7bb2,-7.164150,-34.855460,6
001e7714-7e61-42aa-ad39-d8bf38d4ae0c,-7.154260,-34.850940,6
0023357b-6769-445b-9971-7ad50c068843,-7.029481,-37.283804,0
002e8fa4-f2ea-4c87-9d02-de1d29f21586,-7.262056,-35.934432,3
...,...,...,...
ff9dcd0b-20ad-4f6b-8d75-75d97c0d4346,-7.116305,-34.839329,6
ffa9553f-a98b-45f5-8551-c91a36f9e090,-7.090880,-34.836998,4
ffafc7da-99b9-48e0-951c-c9117bab60c6,-7.083885,-34.843193,4


In [3]:
# Agrupa o DataFrame 'df0' pela coluna 'Cluster' e exbie cálculos dos números de ocorrências (contagem) de cada cluster
grouped_stats = df0.groupby('Cluster').agg({'Cluster': 'count'})

# Exibe as estatísticas agrupadas resultantes
grouped_stats

,Cluster
Cluster,
0,2117
1,91
2,25
3,560
4,4296
5,952
6,9483
7,21


In [4]:
# Dicionário que mapeia cada número de cluster (de 0 a 7) a uma cor específica. As cores são escolhidas para representar a densidade de ativismo de um bairro
cluster_color_dict = {4: 'darkred',     # Nível 7 de ativismo
                      6: 'red',         # Nível 6 de ativismo
                      3: 'orange',      # Nível 5 de ativismo
                      5: 'beige',       # Nível 4 de ativismo
                      2: 'lightgreen',  # Nível 3 de ativismo
                      0: 'lightblue',   # Nível 2 de ativismo
                      1: 'cadetblue',   # Nível 1 de ativismo
                      7: 'darkblue'}    # Nível 0 de ativismo

In [5]:
# Dicionário que mapeia cada cor associada a um cluster para seu respectivo nível de ativismo. As cores estão associadas aos níveis de ativismo em ordem decrescente
cluster_level_dict = {'darkred': 'Nível 7',
                      'red': 'Nível 6',
                      'orange': 'Nível 5',
                      'beige': 'Nível 4',
                      'lightgreen': 'Nível 3',
                      'lightblue': 'Nível 2',
                      'cadetblue': 'Nível 1',
                      'darkblue': 'Nível 0'}

In [ ]:
# Coordenadas do centro do mapa - está configurado para inicializar no CSC da Fundação Cidade Viva
EDI_COORDINATES = (-7.087955846391855, -34.84135519050839)

# Cria um mapa do Folium com o centro nas coordenadas fornecidas e um nível de zoom inicial de 12
map = folium.Map(location=EDI_COORDINATES, zoom_start=12)

# Define o nível mínimo de zoom para o agrupamento e o raio máximo do cluster
minzoom = 20
radius = 50

# Cria clusters de marcadores para diferentes níveis de ativismo, cada um com um nome específico
darkred     = MarkerCluster(overlay=True, control=True, show=True, disableClusteringAtZoom=minzoom, maxClusterRadius=radius, name='Nível 7')
red         = MarkerCluster(overlay=True, control=True, show=True, disableClusteringAtZoom=minzoom, maxClusterRadius=radius, name='Nível 6')
orange      = MarkerCluster(overlay=True, control=True, show=True, disableClusteringAtZoom=minzoom, maxClusterRadius=radius, name='Nível 5')
yellow      = MarkerCluster(overlay=True, control=True, show=True, disableClusteringAtZoom=minzoom, maxClusterRadius=radius, name='Nível 4')
lightgreen  = MarkerCluster(overlay=True, control=True, show=True, disableClusteringAtZoom=minzoom, maxClusterRadius=radius, name='Nível 3')
lightblue   = MarkerCluster(overlay=True, control=True, show=True, disableClusteringAtZoom=minzoom, maxClusterRadius=radius, name='Nível 2')
icyblue     = MarkerCluster(overlay=True, control=True, show=True, disableClusteringAtZoom=minzoom, maxClusterRadius=radius, name='Nível 1')
darkblue    = MarkerCluster(overlay=True, control=True, show=True, disableClusteringAtZoom=minzoom, maxClusterRadius=radius, name='Nível 0')

# Função para retornar o cluster MarkerCluster apropriado com base no nível do cluster
def return_markercluster(cluster):
    level = cluster_level_dict[cluster_color_dict[cluster]]  # Obtém o nível correspondente à cor do cluster
    match(level):  # Usa o match para determinar qual MarkerCluster retornar
        case 'Nível 7':
            return darkred
        case 'Nível 6':
            return red
        case 'Nível 5':
            return orange
        case 'Nível 4':
            return yellow
        case 'Nível 3':
            return lightgreen
        case 'Nível 2':
            return lightblue
        case 'Nível 1':
            return icyblue
        case 'Nível 0':
            return darkblue

# Adiciona marcadores ao mapa, agrupando-os de acordo com o cluster
for i in range(df0.shape[0]):
    lat, lon, cluster = df0.iloc[i].values  # Obtém latitude, longitude e cluster para cada linha do DataFrame
    folium.Marker(location=(lat, lon),  # Cria um marcador no mapa
                  popup=cluster_level_dict[cluster_color_dict[cluster]],  # Define o popup com o nível de ativismo
                  icon=folium.Icon(color=cluster_color_dict[cluster], icon_color='#ffffff', icon='star', prefix='fa')).add_to(return_markercluster(cluster=cluster))

# Adiciona todos os clusters ao mapa
darkred.add_to(map)
red.add_to(map)
orange.add_to(map)
yellow.add_to(map)
lightgreen.add_to(map)
lightblue.add_to(map)
icyblue.add_to(map)
darkblue.add_to(map)

# Adiciona um controle de camada ao mapa para permitir a seleção de quais camadas exibir
folium.LayerControl(collapsed=False).add_to(map)

# Exibe o mapa
display(map)

In [ ]:
# Salva o mapa gerado como um arquivo HTML chamado 'map.html'
map.save('map.html')